In [1]:
import os ; os.chdir(r"D:\Git_repos\Photonic_GCN")
from constants import edges_table
# !pip install torch torch-geometric
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [2]:
def edges_table_to_list(edges_table):
    edges = []
    for key in edges_table.keys():
        nodes_list = edges_table[key]
        for node in nodes_list:
            if ((key, node) not in edges) and ((node, key) not in edges):
                edges.append((key, node))
    return edges

In [3]:
edge_list = edges_table_to_list(edges_table)  # 107 edges
print(len(edge_list)) 
print(edge_list)


107
[('N69', 'N70'), ('N70', 'N71'), ('N71', 'N72'), ('N72', 'N73'), ('N73', 'N74'), ('N73', 'N60'), ('N74', 'N60'), ('N74', 'N94'), ('N60', 'N45'), ('N60', 'N46'), ('N60', 'N47'), ('N60', 'N61'), ('N60', 'N75'), ('N41', 'N42'), ('N42', 'N43'), ('N43', 'N44'), ('N44', 'N45'), ('N45', 'N46'), ('N46', 'N33'), ('N87', 'N86'), ('N86', 'N85'), ('N85', 'N84'), ('N84', 'N83'), ('N83', 'N82'), ('N83', 'N68'), ('N82', 'N68'), ('N82', 'N94'), ('N68', 'N55'), ('N68', 'N54'), ('N68', 'N53'), ('N68', 'N67'), ('N68', 'N81'), ('N59', 'N58'), ('N58', 'N57'), ('N57', 'N56'), ('N56', 'N55'), ('N55', 'N54'), ('N54', 'N34'), ('N33', 'N61'), ('N33', 'N47'), ('N33', 'N48'), ('N34', 'N52'), ('N34', 'N53'), ('N34', 'N67'), ('N94', 'N61'), ('N94', 'N75'), ('N94', 'N76'), ('N95', 'N80'), ('N95', 'N81'), ('N95', 'N82'), ('N95', 'N67'), ('N67', 'N81'), ('N67', 'N53'), ('N67', 'N80'), ('N67', 'N66'), ('N67', 'N52'), ('N53', 'N52'), ('N53', 'N66'), ('N81', 'N80'), ('N81', 'N66'), ('N80', 'N79'), ('N80', 'N66'), ('N

In [4]:
node_names = list(edges_table.keys())  # 51 nodes
print(node_names)

['N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N60', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N87', 'N86', 'N85', 'N84', 'N83', 'N82', 'N68', 'N59', 'N58', 'N57', 'N56', 'N55', 'N54', 'N33', 'N34', 'N94', 'N95', 'N67', 'N53', 'N81', 'N80', 'N52', 'N79', 'N51', 'N65', 'N66', 'N78', 'N64', 'N50', 'N61', 'N75', 'N47', 'N62', 'N76', 'N48', 'N63', 'N77', 'N49']


In [6]:
# Node features (3 features per node)
x = torch.randn((51, 3))  # Random initialization of features for simplicity

# Convert edge list to indices
edge_index = torch.tensor([[node_names.index(u), node_names.index(v)] for u, v in edge_list], dtype=torch.long).t().contiguous()

# Network label (one-hot encoded for 37 categories)
network_label = torch.randint(0, 37, (1,)).item()
y = F.one_hot(torch.tensor(network_label), num_classes=37).float()

# Create data object
data = Data(x=x, edge_index=edge_index, y=y)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # 51 nodes, each with 1 feature from GCN
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        x = self.gcn(data)  # GCN output, shape [num_nodes, 1]
        x = x.view(1, -1)  # Flatten the node features, shape [1, num_nodes]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Output probabilities

# Model parameters
in_channels = 3
hidden_channels = 16
num_classes = 37

# Initialize model
model = Net(in_channels, hidden_channels, num_classes)


In [ ]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(data, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Training the model
train(data, model, optimizer, criterion)
